In [41]:
import torch
import torch.nn.functional as F
from torch.nn.functional import cross_entropy

## **Understanding Architecture**

> **GOAL:** <br>
> Given 3 previous characters, predict 4th character  <br>
> This system will be our base archictecture that generates names

> **NEURAL NETWORK:**  <br>
> 6 embeddings ---> 100 neurons (HIDDEN LAYER) ---> 27 neurons (OUTPUT LAYER) <br>
* We have 6 embeddings because 2 coming from each previous-character<br>
* and 27 neurons in output layer because 26 characters in english + '.' that marks start & end of a name

## **Prepare Dataset**

In [4]:
words = open("names.txt", "r").read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [7]:
# make string to int and reverse mapping
chars = sorted(list(set(''.join(words)))) # gets unique chars (a,b,c...,y,z)
stoi = {s:i for i,s in enumerate(chars)}
stoi['.'] = 0  # '.' char used to mark the start/end of a name
itos = {i:s for s,i in stoi.items()} # reverse mapping

stoi

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25,
 '.': 0}

In [14]:
# Build dataset, first 5 names for now
# we will use 3 context characters to predict next/4th character
# so we are storing X which is the 3 context characters
# and Y, the expected character

block_size=3
X,Y = [],[]
for w in words[:5]:
    print(w+'.')
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in  context), '--->', itos[ix])
        context=context[1:]+[ix]

X=torch.tensor(X)
Y=torch.tensor(Y)

emma.
... ---> e
..e ---> m
.em ---> m
emm ---> .
mm. ---> .
olivia.
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> .
vi. ---> .
ava.
... ---> .
... ---> v
..v ---> .
.v. ---> .
isabella.
... ---> i
..i ---> s
.is ---> .
is. ---> b
s.b ---> e
.be ---> l
bel ---> l
ell ---> .
ll. ---> .
sophia.
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> .
hi. ---> .


In [15]:
X.shape, Y.shape

(torch.Size([32, 3]), torch.Size([32]))

In [16]:
X.dtype, Y.dtype

(torch.int64, torch.int64)

In [21]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  4],
        [ 0,  4, 12],
        [ 4, 12, 12],
        [12, 12,  0],
        [ 0,  0,  0],
        [ 0,  0, 14],
        [ 0, 14, 11],
        [14, 11,  8],
        [11,  8, 21],
        [ 8, 21,  8],
        [21,  8,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0, 21],
        [ 0, 21,  0],
        [ 0,  0,  0],
        [ 0,  0,  8],
        [ 0,  8, 18],
        [ 8, 18,  0],
        [18,  0,  1],
        [ 0,  1,  4],
        [ 1,  4, 11],
        [ 4, 11, 11],
        [11, 11,  0],
        [ 0,  0,  0],
        [ 0,  0, 18],
        [ 0, 18, 14],
        [18, 14, 15],
        [14, 15,  7],
        [15,  7,  8],
        [ 7,  8,  0]])

In [22]:
Y

tensor([ 4, 12, 12,  0,  0, 14, 11,  8, 21,  8,  0,  0,  0, 21,  0,  0,  8, 18,
         0,  1,  4, 11, 11,  0,  0, 18, 14, 15,  7,  8,  0,  0])

## **Understanding Embedding/Lookup Table**

In [31]:
X.shape

torch.Size([32, 3])

In [23]:
C = torch.randn(27,2) # lookup table/embedder

In [26]:
emb = C[X] # embedding process
emb.shape # output shape

torch.Size([32, 3, 2])

In [30]:
# right now emb represents the mapping:  row/record -> index -> embedding
# which is why it has shape [32,3,2]

# we have to now convert this shape  to [32,6] to be able to multiply with weights of neural network
# we do so by using .cat which results in removing the intermediate index
# so now mapping is just:  row/record -> embeddings

torch.cat( [emb[:,0,:], emb[:,1,:], emb[:,2,:]], 1).shape

torch.Size([32, 6])

In [32]:
# this command also does the same thing
emb.view(32,6).shape

torch.Size([32, 6])

In [34]:
#verification
emb.view(32,6) == torch.cat( [emb[:,0,:], emb[:,1,:], emb[:,2,:]], 1)

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, T

In [36]:
# another way, incase we don't know total rows
emb.view(-1,6).shape

torch.Size([32, 6])

## **Neural Network**

In [37]:
# INITIALIZE NETWORK
g = torch.Generator().manual_seed(2147483647) # a fixed seed for consistent results across repeated runs

C = torch.randn((27,2), generator=g) # lookup table.embedder
W1 = torch.randn((6,100), generator=g) # Hidden layer of 100 neurons that accept 6 inputs each
b1 = torch.randn(100, generator=g) # bias
W2 = torch.randn((100,27), generator=g) # Output layer of 27 neurons that accept 100 inputs each
b2 = torch.rand(27, generator=g) # bias
parameters = [C,W1,b1,W2,b2] # trainable parameters

In [38]:
for p in parameters:
    p.requires_grad=True  # enable gradient tracking for the trainable parameters

In [39]:
# stat: number of trainable parameters
sum(p.nelement() for p in parameters)

3481

> Forward Pass -> Loss Calc -> Backward Pass -> Optimize -> Repeat

In [42]:
for _ in range(100):
    # FORWARD PASS
    emb = C[X]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)  # embeddings passed through HIDDEN layer
    logits = h @ W2 + b2 # passed through OUTPUT layer

    # LOSS CALCULATION
    loss = cross_entropy(logits, Y)  # this is equivalent to applying softmax to logits, then doing negative-log-likelihood loss calculation
    print(loss.item())

    # zero_grad
    for p in parameters:
        p.grad = None

    # BACKWARD PASS
    loss.backward()

    # OPTIMIZE TRAINABLE PARAMETERS
    for p in parameters:
        p.data += -0.1 * p.grad

18.49248504638672
13.841938018798828
11.732514381408691
10.12384033203125
8.868001937866211
7.859622478485107
6.975170612335205
6.202770709991455
5.549180030822754
5.004162788391113
4.532743453979492
4.115630149841309
3.741363286972046
3.407202959060669
3.11185359954834
2.8486428260803223
2.6104061603546143
2.3921053409576416
2.1899068355560303
2.0004918575286865
1.8212413787841797
1.6508216857910156
1.4894375801086426
1.3386183977127075
1.2007403373718262
1.078203797340393
0.9725559949874878
0.8844568133354187
0.8139245510101318
0.7594733238220215
0.7177218198776245
0.6850931644439697
0.658933162689209
0.6374107599258423
0.6192907094955444
0.6037342548370361
0.5901567339897156
0.5781320333480835
0.5673421025276184
0.5575427412986755
0.5485435128211975
0.5401943325996399
0.532374918460846
0.5249898433685303
0.5179614424705505
0.5112298727035522
0.5047483444213867
0.4984831213951111
0.49241209030151367
0.48652365803718567
0.480816125869751
0.4752947688102722
0.4699721932411194
0.4648621

## **Now we do above process but for the whole dataset (all names)**

In [64]:
# Build Dataset

block_size=3
X,Y = [],[]
for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context=context[1:]+[ix]

X=torch.tensor(X)
Y=torch.tensor(Y)

In [65]:
# INITIALIZE NETWORK
g = torch.Generator().manual_seed(2147483647) # a fixed seed for consistent results across repeated runs

C = torch.randn((27,2), generator=g) # lookup table.embedder
W1 = torch.randn((6,100), generator=g) # Hidden layer of 100 neurons that accept 6 inputs each
b1 = torch.randn(100, generator=g) # bias
W2 = torch.randn((100,27), generator=g) # Output layer of 27 neurons that accept 100 inputs each
b2 = torch.rand(27, generator=g) # bias
parameters = [C,W1,b1,W2,b2] # trainable parameters

In [66]:
for p in parameters:
    p.requires_grad=True  # enable gradient tracking for the trainable parameters

In [67]:
for _ in range(300):
    # FORWARD PASS
    emb = C[X]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)  # embeddings passed through HIDDEN layer
    logits = h @ W2 + b2 # passed through OUTPUT layer

    # LOSS CALCULATION
    loss = cross_entropy(logits, Y)  # this is equivalent to applying softmax to logits, then doing negative-log-likelihood loss calculation
    print(loss.item())

    # zero_grad
    for p in parameters:
        p.grad = None

    # BACKWARD PASS
    loss.backward()

    # OPTIMIZE TRAINABLE PARAMETERS
    for p in parameters:
        p.data += -0.15 * p.grad

print(f"Final Loss:{loss.item()}")

19.054248809814453
15.494484901428223
13.760018348693848
12.681172370910645
11.859382629394531
11.208608627319336
10.640785217285156
10.124303817749023
9.648913383483887
9.20840072631836
8.798007011413574
8.416089057922363
8.0632963180542
7.740719795227051
7.448310852050781
7.184378147125244
6.945443153381348
6.727380752563477
6.526931285858154
6.342012405395508
6.171337604522705
6.013922691345215
5.868598937988281
5.733917236328125
5.608198165893555
5.4899702072143555
5.37797737121582
5.271376132965088
5.169524192810059
5.0720696449279785
4.978734970092773
4.889416694641113
4.803995132446289
4.7224345207214355
4.644629955291748
4.5705342292785645
4.500037670135498
4.433079719543457
4.369521617889404
4.309239864349365
4.25201416015625
4.197638034820557
4.145840167999268
4.096393585205078
4.049046516418457
4.003616809844971
3.9599061012268066
3.917792558670044
3.8771307468414307
3.8378515243530273
3.7998459339141846
3.763092279434204
3.727499008178711
3.6930925846099854
3.65977668762207